![uc3m](img/uc3m.jpg)

# The min cost flow problem

<a href="http://www.est.uc3m.es/nogales" target="_blank">Javier Nogales</a>



## Summary

Model in Pyomo the min cost flow problem in the Google OR-Tools: https://developers.google.com/optimization/flow/mincostflow

![uc3m](img/mincostflow.png)



    






### Add here the formulation of the problem, including the vector cost and the incidence matrix

You need to solve the problem using the linear-model formulation, i.e. minimize $c'x$ subject to $Ax=b$ and $l\leq x \leq u$ 


## Formulation with Pyomo



### The data



In [2]:
import numpy as np

# The input

start_nodes = [ 0, 0,  1, 1,  1,  2, 2,  3, 4]
end_nodes   = [ 1, 2,  2, 3,  4,  3, 4,  4, 2]
capacities  = [15, 8, 20, 4, 10, 15, 4, 20, 5]
unit_costs  = [ 4, 4,  2, 2,  6,  1, 3,  2, 3]
supplies = [20, 0, 0, -5, -15]

# Incidence matrix (manual form)

#               01 02 12 13 14 23 24  34 42
# A = np.array([[1, 1, 0, 0, 0, 0, 0, 0, 0], # 0
#               [-1,0, 1, 1, 1, 0, 0, 0, 0], # 1
#               [0,-1,-1, 0, 0, 1, 1, 0,-1], # 2 
#               [0, 0, 0,-1, 0,-1, 0, 1, 0], # 3
#               [0, 0, 0, 0,-1, 0,-1,-1, 1], # 4 
#              ])

# Incidence matrix (automatic form)
A = np.zeros((len(supplies),len(start_nodes)))

def generateA (A,start_nodes,end_nodes):
    count = 0
    for i in zip(start_nodes,end_nodes):
        A[i[0]][count] = 1
        A[i[1]][count] = -1
        count+=1

generateA (A,start_nodes,end_nodes) 
c=np.array(unit_costs) # Array with costs
print(c)
b=np.array(supplies) # Array with supplies
print(b)

<function generateA at 0x000001DCA43C20E0>
[4 4 2 2 6 1 3 2 3]
[ 20   0   0  -5 -15]


### The model

In [ ]:
from pyomo.environ import *

# Create an instance of the model
model = ConcreteModel()

# Initialize some ranges for the constraint and Objective definition 
model.r = RangeSet(1,len(A)) #Nodes
model.j = RangeSet(1,len(A.T)) #Connections

'VARIABLE'

#Capacity traspased in each arc
model.cap = Var(model.r,model.j, doc='arc flow used', within=NonNegativeIntegers)

#Just for visualization porpuses
model.gFormat = Var(model.r,model.j, doc='arc flow used', within=Integers) #Needed to be negative for inFlow

'OBJECTIVE'
def cost(model,unit_costs):
    return sum(model.cap[r,j]*A[r-1,j-1]*unit_costs[j-1] for r in model.r for j in model.j if A[r-1,j-1] > 0) #Consider only outflow(Positive Arcs)

model.obj = Objective(rule = cost(model,unit_costs), sense = minimize)

'CONSTRAINTS'
#Respect the maximum capacity of each arc
model.upperLim = ConstraintList()
for r in model.r:
    for j in model.j:
        if A[r-1,j-1] > 0: #To avoid trivial Boolean problem and only counting outflow arcs 
            model.upperLim.add(model.cap[r,j]*A[r-1,j-1] <= capacities[j-1])

#Correct Conections (We have to suggest the algorithm that if 0,1 is produced, then 1,0 is also)
model.correct = ConstraintList()
for j in model.j:
    model.correct.add(sum(model.cap[r,j]*A[r-1,j-1] for r in model.r) == 0) #ENtry and exit in each arc is equal

#Satisfy the supply and the demand of each node 
model.goodSuply = ConstraintList()
for r in model.r: #Each node
    totalFlow = sum(model.cap[r,j]*A[r-1,j-1] for j in model.j) #IN the same row we can find both inFlow and outFlow
    model.goodSuply.add(totalFlow == supplies[r-1])

#Visualization
model.vis = ConstraintList()
for r in model.r:
    for j in model.j:
        model.vis.add(model.cap[r,j]*A[r-1,j-1] == model.gFormat[r,j])

model.pprint()


6 Set Declarations
    cap_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    r*j :   45 : {(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9)}
    correct_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    9 : {1, 2, 3, 4, 5, 6, 7, 8, 9}
    gFormat_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    r*j :   45 : {(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 1), (3, 2), (3, 3), (3, 4

In [ ]:
### Solve the problem

# Define the solver
opt = SolverFactory('gurobi')

# Solve
results = opt.solve(model)  # solve the model with the selected solver
 
model.display()



Model unknown

  Variables:
    cap : arc flow used
        Size=45, Index=cap_index
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (1, 1) :     0 :  12.0 :  None : False : False : NonNegativeIntegers
        (1, 2) :     0 :   8.0 :  None : False : False : NonNegativeIntegers
        (1, 3) :     0 :  None :  None : False :  True : NonNegativeIntegers
        (1, 4) :     0 :  None :  None : False :  True : NonNegativeIntegers
        (1, 5) :     0 :  None :  None : False :  True : NonNegativeIntegers
        (1, 6) :     0 :  None :  None : False :  True : NonNegativeIntegers
        (1, 7) :     0 :  None :  None : False :  True : NonNegativeIntegers
        (1, 8) :     0 :  None :  None : False :  True : NonNegativeIntegers
        (1, 9) :     0 :  None :  None : False :  True : NonNegativeIntegers
        (2, 1) :     0 :  12.0 :  None : False : False : NonNegativeIntegers
        (2, 2) :     0 :  None :  None : False :  True : NonNegativeIntegers
    

### Interpretation

This is the resulting cost of all this transportation is the following:

In [ ]:
print(value(model.obj)) 

150.0


Now let's remove 0s and None values to get a better insight from the result obtained:

In [ ]:
import pandas as pd

#Function to represent properly the names of each variable
def better_representation(dic: dict, name_lists: np.array or list):
    '''Returns a new Dictionary with the key changed in order to obtain insights from data'''
    ini_list = []
    if type(name_lists[0]) == list:
        loops = len(name_lists)
    else:
        loops = 1

    text = ''
    if loops > 1:
        first_elem = name_lists[0]
    else:
        first_elem = name_lists

    for i in first_elem:  #I need to do this crazy thing because the number of loops oscilates from 1 to 3
        if loops > 1:
            for j in name_lists[1]:
                if loops > 2:
                    for k in name_lists[2]:
                        text += '('+str(i)+','+str(j)+','+str(k)+')'
                        ini_list.append(text)
                        text = ''
                else:
                    text += '('+str(i)+','+str(j)+')'
                    ini_list.append(text)
                    text = ''
        else:
            text += str(i)
            ini_list.append(text)
            text = ''

    final_dict = dict(zip(ini_list, list(dic.values())))
    return final_dict


grid_list = []
for (i,j), v in model.gFormat.items():
    if v.stale != True:  #To avoid problems with None values
        grid_list.append(value(v))
    else:
        grid_list.append(0)


grid = {(i): 0  for (i,j) in model.gFormat.items()} #To avoid problems with None values
grid = dict(zip(list(grid.keys()), grid_list))
labels = [['Node0','Node1','Node2','Node3','Node4'],['0,1','0,2','1,2','1,3','1,4','2,3','2,4','3,4','4,2']]
grid = better_representation(grid,labels)
df = pd.DataFrame.from_dict(grid, orient="index", columns=["Number of units"])
df = df[(df > 0).all(1)]
print(df)

             Number of units
(Node0,0,1)             12.0
(Node0,0,2)              8.0
(Node1,1,2)              8.0
(Node1,1,3)              4.0
(Node2,2,3)             12.0
(Node2,2,4)              4.0
(Node3,3,4)             11.0


Based on the above result, the flow distribution was the following:

12 L from 0 to 1

8 L from 0 to 2

8 L from 1 to 2

4 L from 1 to 3

12 L from 2 to 3

4 L from 2 to 4

11 L from 3 to 4

Comparing these result with those that are already in the provided link, both are equal, then it seems that our model is correct.

20 L are the outflow from 0 which makes sense and on the other hand, 5 L is the inflow in 3 (After 11 L are transported from 3 to 4) and 15 is the inflow of 4 so every condition is fulfilled.

During this practice, I get stucked in the 'model.correct' constraint because I did not think about it at the begining that we must equalize the inflow and outflow in each arc, not only to satisfy the supply constraint which is again equalize flows but for each node. 

After I realised, the rest was exactly the same as in the previous extra exercise however considering some details: Now we have a ConcreteModel() which I am more familiar with and the objective function is different, we do not have a sink node where the inflow must be the maximum, now we have to minimize the cost of transportation which is what I did.
